In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [3]:
llm = ChatOpenAI(temperature=0.0, model="gpt-4") 

prompt = ChatPromptTemplate.from_template("Write a short summary about {topic}.")


chain = prompt | llm | StrOutputParser()
result = chain.invoke({"topic": "the history of the internet"})
print(result)

The history of the internet dates back to the 1960s when the U.S. Department of Defense created ARPANET, a project that connected computers at various universities and research labs. In the 1970s, the concept of email was developed and the TCP/IP protocols, which serve as the backbone of the internet communication, were created. The 1980s saw the development of the Domain Name System (DNS) and the birth of the World Wide Web by Tim Berners-Lee in 1989. The 1990s marked the commercialization of the internet with the launch of web browsers like Netscape Navigator and Internet Explorer. The internet has since grown exponentially, becoming an integral part of everyday life, business, and government operations.


In [4]:
# modification 
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
llm = ChatOpenAI(temperature=0.0, model="gpt-4") 


In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])


chain = ({  
        
        "chat_history": lambda x: x.get("chat_history", []),
        "input": lambda x: x["input"]
}
| prompt | llm | StrOutputParser() 
)


chat_history =[
    ("system", "You are a helpful assistant."),
    ("human", "What is the capital of France?"),
    ("assistant", "The capital of France is Paris."),
    ("human", "What is the capital of Germany?"),
    ("assistant", "The capital of Germany is Berlin.")
    
]

result = chain.invoke({"input": "What is the capital of India?", "chat_history": chat_history})


In [6]:
print(result)

The capital of India is New Delhi.


In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableLambda


llm = ChatOpenAI()

In [8]:
summary_chain=( ChatPromptTemplate.from_template("Write a short summary about {text}.") | llm | StrOutputParser()
)

analysis_chain = (ChatPromptTemplate.from_template("Analyze the summary: {text}.") | llm | StrOutputParser())


# parellel LCEL 
combined_chain = {
    "summary": summary_chain,
    "analysis": analysis_chain
} | RunnableLambda(
    lambda x: {
        "summary": x["summary"],
        "analysis": x["analysis"]
    }
)


comined_chain1 = combined_chain.invoke({"text": "the history of the internet"})

In [9]:
print(comined_chain1)

{'summary': 'The history of the internet can be traced back to the 1960s, when researchers developed ARPANET, a network that connected computers at research institutions. This eventually evolved into the modern internet we know today. In the 1980s, the development of TCP/IP protocols standardized communications across different networks, leading to the creation of the World Wide Web in the early 1990s. The internet continued to expand rapidly, with the advent of email, search engines, online commerce, and social media platforms. Today, the internet is an essential part of modern life, connecting people around the world and transforming how information is shared and accessed.', 'analysis': 'The summary provides an overview of the history of the internet, detailing its origins, development, and growth over the years. It likely covers key milestones such as the creation of ARPANET, the development of the World Wide Web, and the expansion of internet access to the general public. The summa

In [10]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableMap,
    RunnableBranch,
    RunnableParallel
)

In [11]:
prompt = PromptTemplate.from_template("Write a short tweet about {topic}.")


In [12]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
parser = StrOutputParser()

In [13]:
add_suffix = RunnableLambda(lambda x: {"topic": x["topic"] + " for developers"})
passthrough = RunnablePassthrough()


In [14]:
multi_topic_map = RunnableMap({
    "ai": RunnableLambda(lambda _: {"topic": "AI"}),
    "ml": RunnableLambda(lambda _: {"topic": "ML"}),
    "cv": RunnableLambda(lambda _: {"topic": "Computer Vision"})
})


In [15]:
full_chain = (
    add_suffix |
    passthrough |
    prompt |
    llm |
    parser
)

In [16]:
full_chain.invoke({"topic": "Python"})

'"Python is a versatile and powerful programming language that makes development easy and enjoyable. #Python #developers"'

In [17]:
print("The End")

The End
